### Machine Translation with Online Learning

In [ ]:
from pathlib import Path # only supported in Python 3
import os

from io_utils import *
from OnlineAlgorithm import *
from TaskModel import *

print("Done.")

#### Parameters

In [ ]:
# EWAF / EXP3 params
algorithm = "EWAF" #"EXP3" # "EWAF
eta_param = 8 # weight update (EWAF)
dp = 2 # reward decimal places
reward_function = "human-avg"
#"human" "human-avg" "human-comet" "comet" "bleu" 

# MT params
lang = "en-de"

# Setup
cv_fold = "all"
run ="1"

data_folder = Path("datasets/{}/".format(lang))
results_folder = Path("results/{}/".format(lang))

#### Data loading

In [ ]:
learn_sent_ids_filepath = data_folder / "shuf_ids.txt"
corpus_filepath = data_folder / "{}.pickle".format(lang)

learning_ids = load_int_list_from_txt(learn_sent_ids_filepath)
print("First:", learning_ids[0])

full_corpus = load_dataframe_from_pickle(corpus_filepath)

print(full_corpus.iloc[0])

print("Done.")

In [ ]:
if lang == "en-de":
    idx = 'sent_id'
else:
    idx = 'sid'

all_model_names = list(full_corpus.system.unique())
all_models = []

num_of_models = len(all_model_names)

for model_name in all_model_names:
    
    system_info = full_corpus.loc[full_corpus['system'] == model_name]
    
    translations = dict(zip(system_info[idx], system_info['mt']))
    human_scores = dict(zip(system_info[idx], system_info['raw_score']))
    bleu_scores = dict(zip(system_info[idx], system_info['bleu_score']))
    comet_scores = dict(zip(system_info[idx], system_info['comet_score']))
    
    model = TaskModel(model_name, translations, human_scores, bleu_scores, comet_scores, num_of_models)
    all_models.append(model)
    
    print(model)
    

#### Online Learning process and evaluation

In [ ]:
oa = init_online_algorithm(algorithm, num_of_models, decimal_places=dp, eta_value=eta_param, reward_function=reward_function)
print(oa)

In [ ]:
weights_filepath = results_folder / "weights_{}_{}_{}_{}_cv{}_{}.csv".format(algorithm, reward_function, dp, eta_param, cv_fold, run)

# print initial weights
models_str = [str(m.model_name) for m in all_models]
weights_str = [str(w) for w in oa.weights_as_probabilities]

with weights_filepath.open("w", encoding="utf8") as f:
    print(','.join(models_str), file=f)
    print(','.join(weights_str), file=f)
    

In [ ]:
t = 1

for sent_id in learning_ids:

    print(" ------------------------- ITERATION {} -------------------------".format(t))  

    f_prediction = oa.forecaster(all_models, sent_id)

    oa.update(all_models, t, sent_id)

    ### print weights
    weights_str = [str(w) for w in oa.weights_as_probabilities]    
    with weights_filepath.open("a", encoding="utf8") as f:
        print(','.join(weights_str), file=f)

    t = t + 1

print("Done.")